In [9]:
import psycopg2
import pandas as pd

# Load cleaned data
df = pd.read_csv("../data/processed/reviews_processed.csv")

print("Available columns:", df.columns.tolist())

# Handle missing columns - add defaults if they don't exist
required_columns = {
    'bank_name': None,  # Required
    'app_name': 'Mobile App',
    'review_text': '',
    'rating': 0,
    'review_date': pd.Timestamp.now().date(),
    'sentiment_label': 'neutral',
    'sentiment_score': 0.0,
    'source': 'unknown'
}

for col, default in required_columns.items():
    if col not in df.columns:
        print(f"Warning: Column '{col}' not found. Adding with default value: {default}")
        if default is None:
            raise ValueError(f"Required column '{col}' is missing from the CSV file!")
        df[col] = default

conn = psycopg2.connect(
    dbname="bank_reviews",
    user="postgres",
    password="12345",  # CHANGE THIS!
    host="localhost",
    port="5432"
)

cur = conn.cursor()

# Insert banks (unique names)
banks = df[['bank_name', 'app_name']].drop_duplicates()

print(f"\nInserting {len(banks)} unique banks...")
for idx, row in banks.iterrows():
    try:
        cur.execute("""
            INSERT INTO banks (bank_name, app_name)
            VALUES (%s, %s) 
            ON CONFLICT (bank_name, app_name) DO NOTHING;
        """, (row['bank_name'], row['app_name']))
    except Exception as e:
        print(f"Error inserting bank {row['bank_name']}: {e}")

conn.commit()

# Create mapping bank_name → bank_id
cur.execute("SELECT bank_id, bank_name FROM banks")
bank_map = {name: bid for bid, name in cur.fetchall()}
print(f"\nFound {len(bank_map)} banks in database")

# Insert reviews
print(f"\nInserting {len(df)} reviews...")
inserted_count = 0
for idx, row in df.iterrows():
    try:
        # Ensure we have a valid bank_id
        bank_id = bank_map.get(row['bank_name'])
        if bank_id is None:
            print(f"Warning: No bank_id found for '{row['bank_name']}'. Skipping review.")
            continue
            
        cur.execute("""
            INSERT INTO reviews
            (bank_id, review_text, rating, review_date, sentiment_label, sentiment_score, source)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (
            bank_id,
            str(row['review_text'])[:5000],  # Limit text length
            float(row['rating']) if pd.notna(row['rating']) else 0,
            pd.to_datetime(row['review_date']).date() if pd.notna(row['review_date']) else pd.Timestamp.now().date(),
            str(row['sentiment_label']),
            float(row['sentiment_score']) if pd.notna(row['sentiment_score']) else 0.0,
            str(row['source'])
        ))
        inserted_count += 1
        
        if inserted_count % 100 == 0:  # Progress indicator
            print(f"Inserted {inserted_count} reviews...")
            
    except Exception as e:
        print(f"Error inserting review {idx}: {e}")
        print(f"Problematic row data: {row.to_dict()}")

conn.commit()
cur.close()
conn.close()

print(f"\nSuccessfully inserted {inserted_count} out of {len(df)} reviews!")

Available columns: ['review_id', 'review_text', 'rating', 'review_date', 'review_year', 'review_month', 'bank_code', 'bank_name', 'user_name', 'thumbs_up', 'text_length', 'source']

Inserting 3 unique banks...
Error inserting bank Bank of Abyssinia: there is no unique or exclusion constraint matching the ON CONFLICT specification

Error inserting bank Commercial Bank of Ethiopia: current transaction is aborted, commands ignored until end of transaction block

Error inserting bank Dashenbank: current transaction is aborted, commands ignored until end of transaction block


Found 3 banks in database

Inserting 1440 reviews...
Inserted 100 reviews...
Inserted 200 reviews...
Inserted 300 reviews...
Inserted 400 reviews...
Inserted 500 reviews...
Inserted 600 reviews...
Inserted 700 reviews...
Inserted 800 reviews...
Inserted 900 reviews...
Inserted 1000 reviews...
Inserted 1100 reviews...
Inserted 1200 reviews...
Inserted 1300 reviews...
Inserted 1400 reviews...

Successfully inserted 1440